In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import pandas as pd
import matplotlib.pyplot as plt

from jigsaw.settings import TRAIN_CSV

df = pd.read_csv(TRAIN_CSV)
df.head()

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0
1,1,SD Stream [ ENG Link 1] (http://www.sportsstre...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,[I wanna kiss you all over! Stunning!](http://...,LOLGA.COM is One of the First Professional Onl...,#Rapper \n🚨Straight Outta Cross Keys SC 🚨YouTu...,[15 Amazing Hidden Features Of Google Search Y...,0
2,2,Lol. Try appealing the ban and say you won't d...,No legal advice: Do not offer or request legal...,pcmasterrace,Don't break up with him or call the cops. If ...,It'll be dismissed: https://en.wikipedia.org/w...,Where is there a site that still works where y...,Because this statement of his is true. It isn'...,1
3,3,she will come your home open her legs with an...,"No Advertising: Spam, referral links, unsolici...",sex,Selling Tyrande codes for 3€ to paypal. PM. \n...,tight pussy watch for your cock get her at thi...,NSFW(obviously) http://spankbang.com/iy3u/vide...,Good News ::Download WhatsApp 2.16.230 APK for...,1
4,4,code free tyrande --->>> [Imgur](http://i.imgu...,"No Advertising: Spam, referral links, unsolici...",hearthstone,wow!! amazing reminds me of the old days.Well...,seek for lady for sex in around http://p77.pl/...,must be watch movie https://sites.google.com/s...,We're streaming Pokemon Veitnamese Crystal RIG...,1


In [18]:
from transformers import AutoTokenizer
from tqdm.notebook import tqdm

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")

In [19]:
def build_samples():
    df = pd.read_csv(TRAIN_CSV)
    rules = df["rule"].unique()
    subreddits = df["subreddit"].unique()

    samples = []

    for row in tqdm(df.itertuples()):
        row_id = row.row_id
        rule = row.rule
        subreddit = row.subreddit
        base_label = int(row.rule_violation)

        for comment_type in [
            "body",
            "positive_example_1",
            "positive_example_2",
            "negative_example_1",
            "negative_example_2",
        ]:
            # Correct label assignment by source field
            if comment_type.startswith("positive"):
                label = 1
            elif comment_type.startswith("negative"):
                label = 0
            else:
                label = base_label

            samples.append(
                {
                    "row_id": row_id,
                    "comment": row.__getattribute__(comment_type),
                    "subreddit": subreddit,
                    "rule": rule,
                    "rule_violation": label,
                }
            )

    return samples


samples = build_samples()

0it [00:00, ?it/s]

In [13]:
unique_samples = []
seen_keys = set()
for sample in samples:
    key = (sample["comment"], sample["rule"], sample["subreddit"])
    if key not in seen_keys:
        unique_samples.append(sample)
        seen_keys.add(key)
len(unique_samples)

7900

In [ ]:
new_df = pd.DataFrame(unique_samples)
new_df.head()

,row_id,comment,subreddit,rule,rule_violation
0,0,Banks don't want you to know this! Click here ...,Futurology,"No Advertising: Spam, referral links, unsolici...",0
1,0,If you could tell your younger self something ...,Futurology,"No Advertising: Spam, referral links, unsolici...",1
2,0,hunt for lady for jack off in neighbourhood ht...,Futurology,"No Advertising: Spam, referral links, unsolici...",1
3,0,Watch Golden Globe Awards 2017 Live Online in ...,Futurology,"No Advertising: Spam, referral links, unsolici...",0
4,0,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",Futurology,"No Advertising: Spam, referral links, unsolici...",0


In [41]:
len(new_df)

7900

In [43]:
from sklearn.model_selection import GroupShuffleSplit

groups = new_df["comment"] + "||" + new_df["rule"]  # composite grouping key
splitter = GroupShuffleSplit(test_size=0.2, random_state=42)
train_idx, test_idx = next(splitter.split(new_df, groups=groups))

train_df, test_df = new_df.iloc[train_idx], new_df.iloc[test_idx]

In [47]:
train_df = train_df.drop_duplicates(subset=["comment", "rule"])
test_df = test_df.drop_duplicates(subset=["comment", "rule"])

print(len(train_df), len(test_df))

1500 375


In [50]:
from datasets import Dataset
from jigsaw.settings import DATA_DIR


def build_dataset(df):
    def build_message(sample):
        return {
            "prompt": [
                {
                    "role": "user",
                    "content": (
                        "You are an unbiased moderator that classifies comments as violating a rule or not. "
                        f"You are moderating for the subreddit <subreddit> r/{sample.subreddit} </subreddit>. "
                        f"The rule is <rule> {sample.rule} </rule>. "
                        "Does the given comment violate the rule? Respond with 'yes' or 'no'."
                        f"Comment: <comment>{sample.comment}</comment>"
                    ),
                }
            ],
            "completion": [
                {
                    "role": "assistant",
                    "content": ("yes" if sample.rule_violation == 1 else "no"),
                }
            ],
        }

    # List of dicts with messages column
    dataset_dicts = [build_message(sample) for sample in df.itertuples()]

    dataset = Dataset.from_list(dataset_dicts)

    return dataset


train_dataset = build_dataset(train_df)
print(f"Dataset size: {len(train_dataset)}")
print(f"Features: {train_dataset.features}")
print(f"\nFirst example:")
print(train_dataset[0])

train_dataset.save_to_disk(DATA_DIR / "ds_no_examples_train.ds")

Dataset size: 1500
Features: {'prompt': List({'content': Value('string'), 'role': Value('string')}), 'completion': List({'content': Value('string'), 'role': Value('string')})}

First example:
{'prompt': [{'content': "You are an unbiased moderator that classifies comments as violating a rule or not. You are moderating for the subreddit <subreddit> r/Futurology </subreddit>. The rule is <rule> No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed. </rule>. Does the given comment violate the rule? Respond with 'yes' or 'no'.Comment: <comment>Banks don't want you to know this! Click here to know more!</comment>", 'role': 'user'}], 'completion': [{'content': 'no', 'role': 'assistant'}]}


Saving the dataset (0/1 shards):   0%|          | 0/1500 [00:00<?, ? examples/s]

In [52]:
test_dataset = build_dataset(test_df)
print(f"Dataset size: {len(test_dataset)}")
print(f"Features: {test_dataset.features}")
print(f"\nFirst example:")
print(test_dataset[0])

test_dataset.save_to_disk(DATA_DIR / "ds_no_examples_eval.ds")

Dataset size: 375
Features: {'prompt': List({'content': Value('string'), 'role': Value('string')}), 'completion': List({'content': Value('string'), 'role': Value('string')})}

First example:
{'prompt': [{'content': 'You are an unbiased moderator that classifies comments as violating a rule or not. You are moderating for the subreddit <subreddit> r/soccerstreams </subreddit>. The rule is <rule> No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed. </rule>. Does the given comment violate the rule? Respond with \'yes\' or \'no\'.Comment: <comment>#Rapper \n🚨Straight Outta Cross Keys SC 🚨YouTube Search Beanie 864 Click Link BELOW To Hear Hit Single\n  "Ah Man" \n Beanie 864 FEAT King Kota \n (King Kota Is Only 15!) Lit 🌡🔥👍💵💯Fr Fr \nhttps://youtu.be/tLqbV1Jmt5Y</comment>', 'role': 'user'}], 'completion': [{'content': 'no', 'role': 'assistant'}]}


Saving the dataset (0/1 shards):   0%|          | 0/375 [00:00<?, ? examples/s]